In [ ]:
! pip install PyPDF2 transformers langchain neo4j faiss-cpu torch

In [ ]:
! pip install langchain huggingface_hub

In [ ]:
! pip install langchain_community

In [ ]:
from langchain.llms import HuggingFaceHub
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
import logging
from typing import Dict, Any
import os
import PyPDF2
from transformers import AutoTokenizer, pipeline
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from neo4j import GraphDatabase
from langchain.llms import HuggingFaceHub
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
import re


In [ ]:
import logging
import sys
from datetime import datetime

def setup_logger(name):
    """Configure logger with both file and console output"""
    # Create logger
    logger = logging.getLogger(name)
    logger.setLevel(logging.DEBUG)

    # Create handlers
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    file_handler = logging.FileHandler(f'debug_{name}_{timestamp}.log')
    console_handler = logging.StreamHandler(sys.stdout)

    # Set levels
    file_handler.setLevel(logging.DEBUG)
    console_handler.setLevel(logging.INFO)

    # Create formatters
    file_formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
    console_formatter = logging.Formatter('%(name)s - %(levelname)s - %(message)s')

    # Add formatters to handlers
    file_handler.setFormatter(file_formatter)
    console_handler.setFormatter(console_formatter)

    # Add handlers to logger
    logger.addHandler(file_handler)
    logger.addHandler(console_handler)

    return logger

In [ ]:
from transformers import pipeline
import PyPDF2
import logging

class TextProcessor:
    def __init__(self):
        self.logger = setup_logger('TextProcessor')
        self.logger.info("Initializing TextProcessor")
        try:
            # Initialize the Named Entity Recognition model
            self.ner_pipeline = pipeline("ner", model="dslim/bert-base-NER")
            self.logger.info("NER pipeline initialized successfully")
        except Exception as e:
            self.logger.error(f"Failed to initialize NER pipeline: {str(e)}")
            raise
# Extraction
    def extract_text_from_pdf(self, pdf_path): # Open PDF in binary read mode
        """Extract text content from PDF file"""
        self.logger.info(f"Starting text extraction from {pdf_path}")
        text = "" # Initialize empty string to store text
        try:
            with open(pdf_path, 'rb') as file:
                pdf_reader = PyPDF2.PdfReader(file)
                total_pages = len(pdf_reader.pages)
                self.logger.debug(f"PDF has {total_pages} pages")

                # Loop through each page
                for i, page in enumerate(pdf_reader.pages):
                    self.logger.debug(f"Processing page {i+1}/{total_pages}")
                    page_text = page.extract_text()  # Extract text from current page
                    text += page_text         # Add to total text
                    self.logger.debug(f"Page {i+1} extracted: {len(page_text)} characters")

            self.logger.info(f"Text extraction completed. Total length: {len(text)} characters")
            return text
        except Exception as e:
            self.logger.error(f"Error extracting text from PDF: {str(e)}")
            raise
# NER
    def perform_ner(self, text):
      """Perform Named Entity Recognition on text specifically for fertility domain"""
      self.logger.info("Starting fertility domain NER processing")
      try:
          self.logger.debug(f"Input text length: {len(text)} characters")
          ner_results = self.ner_pipeline(text)
          self.logger.debug(f"Found {len(ner_results)} entity mentions")

          # Initialize result categories
          entities = {
              'CONSEQUENCE': set(),      # Consequences
              'TREATMENT': set(),        # Treatment methods and protocols
              'MEDICATION': set(),       # Medications and hormones

          }

          # # Define domain-specific terms to look for : Dictionary of domain-specific terms
          fertility_terms = {
              'CONSEQUENCE': [                                              #category
                  'infertility', 'subfertility', 'PCOS', 'endometriosis',   #terms
                  'amenorrhea', 'oligospermia', 'azoospermia', 'anovulation', #one term
                  'recurrent miscarriage', 'uterine fibroids'
              ],
              'TREATMENT': [
                  'IVF', 'ICSI', 'IUI', 'ovulation induction',
                  'hormone therapy', 'fertility preservation',
                  'embryo transfer', 'assisted hatching'
              ],
              'MEDICATION': [
                  'Clomid', 'Letrozole', 'FSH', 'hCG', 'progesterone',
                  'Gonal-F', 'Menopur', 'Ovidrel', 'Lupron'
               ]
          }

          # Direct matching : Search for each term in each category
          for category, terms in fertility_terms.items():
              for term in terms:
                  # # Create case-insensitive pattern
                  pattern = re.compile(r'\b' + re.escape(term) + r'\b', re.IGNORECASE)
                  matches = pattern.finditer(text)
                  for match in matches:
                      entities[category].add(term.lower())
                      self.logger.debug(f"Found {category}: {term}")

          # Additional context-based extraction
          patterns = {
              'CONSEQUENCE': r'consequence with ([^.]*)',
              'TREATMENT': r'treatment (?:with|using) ([^.]*)',
              'MEDICATION': r'medication ([^.]*)'

          }

          for entity_type, pattern in patterns.items():
              matches = re.finditer(pattern, text, re.IGNORECASE)
              for match in matches:
                  entity = match.group(1).strip().lower()
                  entities[entity_type].add(entity)
                  self.logger.debug(f"Found contextual {entity_type}: {entity}")

          # Log results
          for entity_type, values in entities.items():
              self.logger.info(f"Total {entity_type} entities found: {len(values)}")
              self.logger.debug(f"{entity_type} entities: {values}")

          return entities

      except Exception as e:
          self.logger.error(f"Error during fertility NER processing: {str(e)}")
          raise



In [ ]:
from neo4j import GraphDatabase
import logging

class Neo4jHandler:

# Connection to DB
    def __init__(self, uri, user, password):
        self.logger = setup_logger('Neo4jHandler')
        self.logger.info("Initializing Neo4j connection")
        try:
            # Create connection to Neo4j database
            self.driver = GraphDatabase.driver(uri, auth=(user, password))
            self.logger.info("Connected to Neo4j successfully")
            # Set up database constraints
            self._create_constraints()
        except Exception as e:
            self.logger.error(f"Failed to initialize Neo4j connection: {str(e)}")
            raise

# create constraints
    def _create_constraints(self):
        """Create constraints for unique entity names"""
        self.logger.info("Creating database constraints")
        with self.driver.session() as session:
            try:
                # Define constraints for each entity type
                constraints = [
                    "CREATE CONSTRAINT IF NOT EXISTS FOR (c:CONSEQUENCE) REQUIRE c.name IS UNIQUE",
                    "CREATE CONSTRAINT IF NOT EXISTS FOR (t:TREATMENT) REQUIRE t.name IS UNIQUE",
                    "CREATE CONSTRAINT IF NOT EXISTS FOR (m:MEDICATION) REQUIRE m.name IS UNIQUE"
                ]

                # Execute each constraint
                for constraint in constraints:
                    self.logger.debug(f"Executing constraint: {constraint}")
                    session.run(constraint)
                self.logger.info("All constraints created successfully")
            except Exception as e:
                self.logger.error(f"Error creating constraints: {str(e)}")
                raise

# storing entities in the DB
    def store_entities(self, entities):
        """Store entities in Neo4j database"""
        self.logger.info("Starting entity storage")
        total_stored = 0

        with self.driver.session() as session:
            # Process each entity type (CONSEQUENCE, TREATMENT, MEDICATION)
            for entity_type, values in entities.items():
                self.logger.debug(f"Processing {len(values)} {entity_type} entities")

                for value in values:
                    try:
                        # Create appropriate query based on entity type
                        if entity_type == 'CONSEQUENCE':
                            query = "MERGE (c:CONSEQUENCE {name: $name})"
                        elif entity_type == 'TREATMENT':
                            query = "MERGE (t:TREATMENT {name: $name})"
                        elif entity_type == 'MEDICATION':
                            query = "MERGE (m:MEDICATION {name: $name})"

                        self.logger.debug(f"Executing query: {query} with name={value}")
                        session.run(query, name=value)
                        total_stored += 1

                    except Exception as e:
                        self.logger.error(f"Error storing entity {value} of type {entity_type}: {str(e)}")

        self.logger.info(f"Entity storage completed. Total entities stored: {total_stored}")
# Close DB connection
    def close(self):
        """Close the Neo4j connection"""
        self.logger.info("Closing Neo4j connection")
        try:
            self.driver.close()
            self.logger.info("Neo4j connection closed successfully")
        except Exception as e:
            self.logger.error(f"Error closing Neo4j connection: {str(e)}")
            raise

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
import logging

class ChunkRetriever:
    def __init__(self):
        self.logger = setup_logger('ChunkRetriever')
        self.logger.info("Initializing ChunkRetriever")

        try:
            # Initialize text splitter with specific parameters
            self.text_splitter = RecursiveCharacterTextSplitter(
                chunk_size=1000,
                chunk_overlap=200,
                length_function=len
            )
            self.logger.debug("Text splitter initialized with chunk_size=1000, overlap=200")

            # Initialize HuggingFace embeddings for vector conversion
            self.embeddings = HuggingFaceEmbeddings()
            self.logger.debug("HuggingFace embeddings initialized")

            # Vector store starts as None until created
            self.vector_store = None
            self.logger.info("ChunkRetriever initialized successfully")
        except Exception as e:
            self.logger.error(f"Error initializing ChunkRetriever: {str(e)}")
            raise

# chunking
    def create_chunks(self, text):
        """Split text into chunks"""
        self.logger.info("Starting text chunking")
        self.logger.debug(f"Input text length: {len(text)} characters")

        try:
            # Split text into overlapping chunks
            chunks = self.text_splitter.create_documents([text])
            self.logger.info(f"Created {len(chunks)} chunks")

            # Log details about each chunk
            for i, chunk in enumerate(chunks):
                self.logger.debug(f"Chunk {i+1} length: {len(chunk.page_content)} characters")

            return chunks
        except Exception as e:
            self.logger.error(f"Error creating chunks: {str(e)}")
            raise

# Vector store creation
    def create_vector_store(self, chunks):
        """Create vector store from text chunks"""
        self.logger.info("Creating vector store")
        try:
            # Convert text chunks to vector embeddings and store in FAISS
            self.vector_store = FAISS.from_documents(chunks, self.embeddings)
            self.logger.info("Vector store created successfully")
            return self.vector_store
        except Exception as e:
            self.logger.error(f"Error creating vector store: {str(e)}")
            raise

# Retrieve chunks
    def retrieve_relevant_chunks(self, query, k=3):
        """Retrieve relevant chunks for a given query"""
        self.logger.info(f"Retrieving chunks for query: {query}")

        # Check if vector store exists
        if self.vector_store is None:
            self.logger.error("Vector store not initialized")
            raise ValueError("Vector store not initialized. Process a document first.")

        try:
            # Find k most similar chunks to query
            relevant_chunks = self.vector_store.similarity_search(query, k=k)
            self.logger.info(f"Retrieved {len(relevant_chunks)} chunks")

            # Log details about retrieved chunks
            for i, chunk in enumerate(relevant_chunks):
                self.logger.debug(f"Chunk {i+1} length: {len(chunk.page_content)} characters")

            return relevant_chunks
        except Exception as e:
            self.logger.error(f"Error retrieving chunks: {str(e)}")
            raise

In [ ]:
from langchain.llms import HuggingFaceHub
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
import logging
from langchain.llms import HuggingFaceHub
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
import logging
from typing import Dict, Any
import os

class QuestionAnswerer:
    def __init__(self, hf_token: str):
        self.logger = setup_logger('QuestionAnswerer')
        self.logger.info("Initializing QuestionAnswerer")

        try:
            # Set HuggingFace token in environment variable
            os.environ["HUGGINGFACE_API_TOKEN"] = hf_token

            # Initialize Mistral model
            self.llm = HuggingFaceHub(
                repo_id="mistralai/Mistral-7B-Instruct-v0.3",
                huggingfacehub_api_token=hf_token,
                model_kwargs={
                    "temperature": 0.1,     # Low temperature for more focused answers
                    "max_new_tokens": 512,  # Maximum length of generated response
                    "top_p": 0.95           # Nucleus sampling parameter
                }
            )
            self.logger.debug("Mistral model initialized")

# Prompt Template Setup
            self.qa_prompt = PromptTemplate(
                input_variables=["context", "question"],
                template="""
                Based on the following context, please answer the question.
                If the answer cannot be found in the context, say "I cannot find information about this in the provided context."

                Context: {context}

                Question: {question}

                Answer:"""
            )
            self.logger.debug("Prompt template created")
# QA Chain Creation
            self.qa_chain = LLMChain(
                llm=self.llm,
                prompt=self.qa_prompt
            )
            self.logger.info("QA chain initialized successfully")

        except Exception as e:
            self.logger.error(f"Error initializing QuestionAnswerer: {str(e)}")
            raise

    def answer_question(self, question: str, context: str) -> str:
        """Generate answer using QA chain"""
        self.logger.info(f"Processing question: {question}")
        self.logger.debug(f"Context length: {len(context)} characters")

        try:
            # Verify token exists
            if not os.environ.get("HUGGINGFACE_API_TOKEN"):
                raise ValueError("HuggingFace API token not found in environment variables")

            # Generate answer
            response = self.qa_chain.invoke({
                "context": context,
                "question": question
            })

            # Extract answer from response
            answer = response.get("text", "Error: No response generated")

            self.logger.info("Answer generated successfully")
            self.logger.debug(f"Answer: {answer}")
            return answer

        except Exception as e:
            error_msg = f"Error generating answer: {str(e)}"
            self.logger.error(error_msg)
            return f"Error: {error_msg}"

In [ ]:
def test_huggingface_token(token: str) -> bool:
    """Test if the HuggingFace token is valid"""
    import requests

    headers = {"Authorization": f"Bearer {token}"}
    api_url = "https://api-inference.huggingface.co/models/mistralai/Mistral-7B-Instruct-v0.3"

    try:
        response = requests.get(api_url, headers=headers)
        if response.status_code == 200:
            return True
        elif response.status_code == 401:
            print(f"Token authentication failed. Status code: {response.status_code}")
            return False
        else:
            print(f"API request failed. Status code: {response.status_code}")
            return False
    except Exception as e:
        print(f"Error testing token: {str(e)}")
        return False

In [ ]:
import logging # For creating log messages
import time     # For timing operations
from datetime import datetime # For working with dates and times
import sys # For system-specific functions
import traceback # For detailed error tracking

def format_time(seconds):
    """Format execution time in a human-readable format"""
    minutes, seconds = divmod(seconds, 60)
    hours, minutes = divmod(minutes, 60)
    return f"{int(hours):02d}:{int(minutes):02d}:{seconds:05.2f}"

def main():
    # Setup main logger
    logger = setup_logger('MainScript')
    start_time = time.time()

    # Configuration
    NEO4J_URI =
    NEO4J_USER = "neo4j"
    NEO4J_PASSWORD =
    HF_TOKEN =
    PDF_PATH = "/content/Pub102023.pdf"


    if test_huggingface_token(HF_TOKEN):
        print("Token is valid!")
        qa_system = QuestionAnswerer(HF_TOKEN)
    else:
        print("Invalid token. Please check your HuggingFace token.")

    # Initialize components
    text_processor = None
    neo4j_handler = None
    chunk_retriever = None
    qa_system = None

    try:
        logger.info("=== Starting Document Processing Pipeline ===")
        logger.debug(f"Configuration:\n"
                    f"PDF Path: {PDF_PATH}\n"
                    f"Neo4j URI: {NEO4J_URI}")

        # Initialize components with timing
        logger.info("Initializing components...")
        init_start = time.time()

        text_processor = TextProcessor()
        logger.debug("TextProcessor initialized")

        neo4j_handler = Neo4jHandler(NEO4J_URI, NEO4J_USER, NEO4J_PASSWORD)
        logger.debug("Neo4jHandler initialized")

        chunk_retriever = ChunkRetriever()
        logger.debug("ChunkRetriever initialized")

        qa_system = QuestionAnswerer(HF_TOKEN)
        logger.debug("QuestionAnswerer initialized")

        init_time = time.time() - init_start
        logger.info(f"All components initialized successfully in {format_time(init_time)}")

        # Extract text from PDF
        logger.info("Starting PDF text extraction...")
        pdf_start = time.time()
        text = text_processor.extract_text_from_pdf(PDF_PATH)
        pdf_time = time.time() - pdf_start
        logger.info(f"Text extraction completed in {format_time(pdf_time)}")
        logger.debug(f"Extracted text length: {len(text)} characters")

        # Perform NER
        logger.info("Starting Named Entity Recognition...")
        ner_start = time.time()
        entities = text_processor.perform_ner(text)
        ner_time = time.time() - ner_start
        logger.info(f"NER completed in {format_time(ner_time)}")

        entity_stats = {k: len(v) for k, v in entities.items()}
        logger.info(f"Entity statistics: {entity_stats}")

        # Store entities in Neo4j
        logger.info("Starting Neo4j storage...")
        neo4j_start = time.time()
        neo4j_handler.store_entities(entities)
        neo4j_time = time.time() - neo4j_start
        logger.info(f"Neo4j storage completed in {format_time(neo4j_time)}")

        # Create chunks and vector store
        logger.info("Starting text chunking and vector store creation...")
        chunk_start = time.time()
        chunks = chunk_retriever.create_chunks(text)
        logger.debug(f"Created {len(chunks)} chunks")

        vector_store = chunk_retriever.create_vector_store(chunks)
        chunk_time = time.time() - chunk_start
        logger.info(f"Chunking and vector store creation completed in {format_time(chunk_time)}")

        # Example question answering
        logger.info("Testing question answering system...")
        qa_start = time.time()

        test_questions = [
            "What is infertility swhich this document?",
            "What is the summary of this document?"
        ]
# Question Answering
        for question in test_questions:
            logger.info(f"\nProcessing question: {question}")

            # Retrieve relevant chunks
            relevant_chunks = chunk_retriever.retrieve_relevant_chunks(question)
            context = " ".join([chunk.page_content for chunk in relevant_chunks])
            logger.debug(f"Retrieved context length: {len(context)} characters")

            # Generate answer
            answer = qa_system.answer_question(question, context)
            logger.info(f"Question: {question}")
            logger.info(f"Answer: {answer}")

        qa_time = time.time() - qa_start
        logger.info(f"Question answering testing completed in {format_time(qa_time)}")

        # Calculate and log total execution time
        total_time = time.time() - start_time
        logger.info("\n=== Processing Summary ===")
        logger.info(f"Initialization time: {format_time(init_time)}")
        logger.info(f"PDF extraction time: {format_time(pdf_time)}")
        logger.info(f"NER processing time: {format_time(ner_time)}")
        logger.info(f"Neo4j storage time: {format_time(neo4j_time)}")
        logger.info(f"Chunking and vectorization time: {format_time(chunk_time)}")
        logger.info(f"Question answering time: {format_time(qa_time)}")
        logger.info(f"Total execution time: {format_time(total_time)}")
        logger.info("=== Pipeline completed successfully ===")

    except Exception as e:
        logger.error("=== Pipeline failed ===")
        logger.error(f"Error: {str(e)}")
        logger.error("Traceback:")
        logger.error(traceback.format_exc())
        raise

    finally:
        logger.info("Cleaning up resources...")
        try:
            # Clean up Neo4j connection
            if neo4j_handler:
                neo4j_handler.close()
                logger.debug("Neo4j connection closed")

            # Clean up other resources if needed
            logger.info("Cleanup completed")

        except Exception as cleanup_error:
            logger.error(f"Error during cleanup: {str(cleanup_error)}")
            logger.error(traceback.format_exc())

if __name__ == "__main__":
    try:
        main()
    except Exception as e:
        print(f"Critical error: {str(e)}")
        sys.exit(1)

Token is valid!
QuestionAnswerer - INFO - Initializing QuestionAnswerer
QuestionAnswerer - INFO - Initializing QuestionAnswerer
QuestionAnswerer - INFO - Initializing QuestionAnswerer
QuestionAnswerer - INFO - Initializing QuestionAnswerer
QuestionAnswerer - INFO - Initializing QuestionAnswerer
QuestionAnswerer - INFO - Initializing QuestionAnswerer
QuestionAnswerer - INFO - Initializing QuestionAnswerer
QuestionAnswerer - INFO - Initializing QuestionAnswerer
QuestionAnswerer - INFO - Initializing QuestionAnswerer
QuestionAnswerer - INFO - Initializing QuestionAnswerer
QuestionAnswerer - INFO - Initializing QuestionAnswerer
QuestionAnswerer - INFO - Initializing QuestionAnswerer
QuestionAnswerer - INFO - Initializing QuestionAnswerer
QuestionAnswerer - INFO - Initializing QuestionAnswerer
QuestionAnswerer - INFO - Initializing QuestionAnswerer


INFO:QuestionAnswerer:Initializing QuestionAnswerer
DEBUG:QuestionAnswerer:Mistral model initialized
DEBUG:QuestionAnswerer:Prompt template created


QuestionAnswerer - INFO - QA chain initialized successfully
QuestionAnswerer - INFO - QA chain initialized successfully
QuestionAnswerer - INFO - QA chain initialized successfully
QuestionAnswerer - INFO - QA chain initialized successfully
QuestionAnswerer - INFO - QA chain initialized successfully
QuestionAnswerer - INFO - QA chain initialized successfully
QuestionAnswerer - INFO - QA chain initialized successfully
QuestionAnswerer - INFO - QA chain initialized successfully
QuestionAnswerer - INFO - QA chain initialized successfully
QuestionAnswerer - INFO - QA chain initialized successfully
QuestionAnswerer - INFO - QA chain initialized successfully
QuestionAnswerer - INFO - QA chain initialized successfully
QuestionAnswerer - INFO - QA chain initialized successfully
QuestionAnswerer - INFO - QA chain initialized successfully
QuestionAnswerer - INFO - QA chain initialized successfully


INFO:QuestionAnswerer:QA chain initialized successfully


MainScript - INFO - === Starting Document Processing Pipeline ===
MainScript - INFO - === Starting Document Processing Pipeline ===
MainScript - INFO - === Starting Document Processing Pipeline ===
MainScript - INFO - === Starting Document Processing Pipeline ===
MainScript - INFO - === Starting Document Processing Pipeline ===
MainScript - INFO - === Starting Document Processing Pipeline ===
MainScript - INFO - === Starting Document Processing Pipeline ===
MainScript - INFO - === Starting Document Processing Pipeline ===


...

INFO:Neo4jHandler:Closing Neo4j connection


Neo4jHandler - INFO - Neo4j connection closed successfully
Neo4jHandler - INFO - Neo4j connection closed successfully
Neo4jHandler - INFO - Neo4j connection closed successfully
Neo4jHandler - INFO - Neo4j connection closed successfully
Neo4jHandler - INFO - Neo4j connection closed successfully
Neo4jHandler - INFO - Neo4j connection closed successfully
Neo4jHandler - INFO - Neo4j connection closed successfully
Neo4jHandler - INFO - Neo4j connection closed successfully


INFO:Neo4jHandler:Neo4j connection closed successfully
DEBUG:MainScript:Neo4j connection closed


MainScript - INFO - Cleanup completed
MainScript - INFO - Cleanup completed
MainScript - INFO - Cleanup completed
MainScript - INFO - Cleanup completed
MainScript - INFO - Cleanup completed
MainScript - INFO - Cleanup completed
MainScript - INFO - Cleanup completed
MainScript - INFO - Cleanup completed


INFO:MainScript:Cleanup completed
